In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
IDS2017URL = "/gdrive/MyDrive/IDS2017/IDS2017.csv"


In [3]:
import torch
import pandas as pd
import tensorflow as tf
import numpy as np
import torch.nn as nn
from numpy import vstack
from numpy import argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Softmax
from torch.nn import Module
from torch.optim import SGD
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
from torchvision import transforms

In [4]:
torch.random.manual_seed(2) 

In [8]:
class IDSDataset(Dataset):

    def __init__(self, IDS2017URL, tranform=True):
      df = pd.read_csv(IDS2017URL)
      df = df[~df['Label'].isin(['Heartbleed', 'Web Attack - Sql Injection', 'Infiltration', 'Web Attack - XSS', 'Web Attack - Brute Force'])]
      df = df.replace([-np.inf, np.inf], np.nan)
      df = df.dropna()
      df.drop(df.columns[[37,39,61,62,63,64,65,66]],axis=1,inplace=True)
      self.X = df.iloc[:, :-2].apply(lambda x: (x-np.mean(x))/np.std(x)).values
      self.y = df.values[:, -2]
      self.X = self.X.astype('float32')
      le = LabelEncoder()
      le.fit(["BENIGN", "DoS Hulk", "PortScan", "DDoS", "DoS GoldenEye","FTP-Patator","SSH-Patator","DoS slowloris","DoS Slowhttptest","Bot"])
      self.y = le.transform(self.y)


    def __len__(self):
      return len(self.X)
    
    def __getitem__(self, idx):
      return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0.33):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 76)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(76, 76)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(76, 10)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)

    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)

        # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)

        # output layer
        X = self.hidden3(X)
        X = self.act3(X)

        return X
 
def prepare_data(path):
    # load the dataset
    dataset = IDSDataset(path)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=1000, shuffle=True)
    test_dl = DataLoader(test, batch_size=1000, shuffle=False)
    return train_dl, test_dl

def train_model(train_dl, model):
    # define the optimization
    criterion = CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=0.0001)
    # enumerate epochs
    for epoch in range(10):
        # enumerate mini batches
        print(model.state_dict())
        for i, (inputs, targets) in enumerate(train_dl):
            # clear the gradients
            optimizer.zero_grad()
            #torch.autograd.set_detect_anomaly(True)
            # compute the model output
            #print(inputs)
            #print("******************")
            yhat = model(inputs)
            #print(yhat)
            # calculate loss
            loss = criterion(yhat, targets)
            # credit assignment
            loss.backward()
            # update model weights
            optimizer.step()
            print('[%d, %5d] loss: %.3f' %(epoch + 1, i + 1, loss.item()))
            running_loss = 0.0

def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        # evaluate the model on the test set
        yhat = model(inputs)
        # retrieve numpy array
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        # convert to class labels
        yhat = argmax(yhat, axis=1)
        # reshape for stacking
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        # store
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    # calculate accuracy
    acc = accuracy_score(actuals, predictions)
    return acc

def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [109]:
# class IDSDataset(Dataset):

#     def __init__(self, IDS2017URL, tranform=True):
#       df = pd.read_csv(IDS2017URL)
#       df = df[~df['Label'].isin(['Heartbleed', 'Web Attack - Sql Injection', 'Infiltration', 'Web Attack - XSS', 'Web Attack - Brute Force'])]
#       df = df.replace([-np.inf, np.inf], np.nan)
#       df = df.dropna()
#       df = pd.concat([df.iloc[:,0:37], df[:,38:],df.iloc[:, -2:]], axis=1)
#       self.X = df.iloc[:, :-2].apply(lambda x: (x-np.mean(x))/np.std(x)+0.00001).values
#       self.y = df.values[:, -2]
#       self.X = self.X.astype('float32')
#       self.y = LabelEncoder().fit_transform(self.y)

#     def __len__(self):
#       return len(self.X)
    
#     def __getitem__(self, idx):
#       return [self.X[idx], self.y[idx]]
 
#     # get indexes for train and test rows
#     def get_splits(self, n_test=0.33):
#         # determine sizes
#         test_size = round(n_test * len(self.X))
#         train_size = len(self.X) - test_size
#         # calculate the split
#         return random_split(self, [train_size, test_size])

# class MLP(Module):
#     # define model elements
#     def __init__(self, n_inputs):
#         super(MLP, self).__init__()
#         # input to first hidden layer
#         self.hidden1 = Linear(n_inputs, 76)
#         #kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
#         self.act1 = ReLU()
#         # second hidden layer
#         self.hidden2 = Linear(76, 76)
#         #kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
#         self.act2 = ReLU()
#         # third hidden layer and output
#         self.hidden3 = Linear(76, 10)
#         #xavier_uniform_(self.hidden3.weight)
#         self.act3 = Softmax(dim=1)

#     def forward(self, X):
#         # input to first hidden layer
#         X = self.hidden1(X)
#         X = self.act1(X)
#         print(X)
#         # second hidden layer
#         X = self.hidden2(X)
#         X = self.act2(X)
#         print(X)
#         # output layer
#         X = self.hidden3(X)
#         X = self.act3(X)
#         print(X)
#         return X
 
# def prepare_data(path):
#     # load the dataset
#     dataset = IDSDataset(path)
#     # calculate split
#     train, test = dataset.get_splits()
#     # prepare data loaders
#     train_dl = DataLoader(train, batch_size=1000, shuffle=True)
#     test_dl = DataLoader(test, batch_size=1000, shuffle=False)
#     return train_dl, test_dl

# def train_model(train_dl, model):
#     # define the optimization
#     criterion = CrossEntropyLoss()
#     optimizer = Adam(model.parameters(), lr=0.0001)
#     # enumerate epochs
#     for epoch in range(10):
#         # enumerate mini batches
#         print(model.state_dict())
#         for i, (inputs, targets) in enumerate(train_dl):
#             # clear the gradients
#             optimizer.zero_grad()
#             #torch.autograd.set_detect_anomaly(True)
#             # compute the model output
#             #print(inputs)
#             #print("******************")
#             yhat = model(inputs)
#             #print(yhat)
#             # calculate loss
#             loss = criterion(yhat, targets)
#             # credit assignment
#             loss.backward()
#             # update model weights
#             optimizer.step()

# def evaluate_model(test_dl, model):
#     predictions, actuals = list(), list()
#     for i, (inputs, targets) in enumerate(test_dl):
#         # evaluate the model on the test set
#         yhat = model(inputs)
#         # retrieve numpy array
#         yhat = yhat.detach().numpy()
#         actual = targets.numpy()
#         # convert to class labels
#         yhat = argmax(yhat, axis=1)
#         # reshape for stacking
#         actual = actual.reshape((len(actual), 1))
#         yhat = yhat.reshape((len(yhat), 1))
#         # store
#         predictions.append(yhat)
#         actuals.append(actual)
#     predictions, actuals = vstack(predictions), vstack(actuals)
#     # calculate accuracy
#     acc = accuracy_score(actuals, predictions)
#     return acc

# def predict(row, model):
#     # convert row to data
#     row = Tensor([row])
#     # make prediction
#     yhat = model(row)
#     # retrieve numpy array
#     yhat = yhat.detach().numpy()
#     return yhat

In [9]:
train_dl, test_dl = prepare_data(IDS2017URL)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
train_dl.dataset

In [10]:
print(len(train_dl.dataset), len(test_dl.dataset))

1893185 932464


In [11]:
model = MLP(75)

In [12]:
train_model(train_dl, model)

Streaming output truncated to the last 5000 lines.
[8,  1065] loss: 1.567
[8,  1066] loss: 1.575
[8,  1067] loss: 1.559
[8,  1068] loss: 1.584
[8,  1069] loss: 1.572
[8,  1070] loss: 1.559
[8,  1071] loss: 1.546
[8,  1072] loss: 1.580
[8,  1073] loss: 1.571
[8,  1074] loss: 1.564
[8,  1075] loss: 1.570
[8,  1076] loss: 1.587
[8,  1077] loss: 1.567
[8,  1078] loss: 1.580
[8,  1079] loss: 1.580
[8,  1080] loss: 1.559
[8,  1081] loss: 1.574
[8,  1082] loss: 1.572
[8,  1083] loss: 1.572
[8,  1084] loss: 1.561
[8,  1085] loss: 1.567
[8,  1086] loss: 1.575
[8,  1087] loss: 1.582
[8,  1088] loss: 1.558
[8,  1089] loss: 1.565
[8,  1090] loss: 1.580
[8,  1091] loss: 1.569
[8,  1092] loss: 1.580
[8,  1093] loss: 1.568
[8,  1094] loss: 1.573
[8,  1095] loss: 1.569
[8,  1096] loss: 1.559
[8,  1097] loss: 1.561
[8,  1098] loss: 1.567
[8,  1099] loss: 1.578
[8,  1100] loss: 1.583
[8,  1101] loss: 1.580
[8,  1102] loss: 1.574
[8,  1103] loss: 1.561
[8,  1104] loss: 1.553
[8,  1105] loss: 1.579
[8,  1

In [14]:
torch.save(model.state_dict(), '/gdrive/MyDrive/IDS2017/3.pth')

In [13]:
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

Accuracy: 0.892


In [15]:
print(model.state_dict())

OrderedDict([('hidden1.weight', tensor([[-0.2239, -0.2960,  0.0677,  ...,  0.0122, -0.2374, -0.1845],
        [-0.1533, -0.4737,  0.2029,  ..., -0.1123, -0.1179,  0.1332],
        [ 0.0707, -0.0355,  0.1494,  ...,  0.1412, -0.2020,  0.0969],
        ...,
        [ 0.2198, -0.2857, -0.0925,  ..., -0.1318, -0.0788, -0.1972],
        [ 0.3144, -0.4556,  0.0799,  ..., -0.0896,  0.0063,  0.0856],
        [-0.1432, -0.3922,  0.2898,  ..., -0.2016,  0.2073,  0.1498]])), ('hidden1.bias', tensor([ 0.0996,  0.0590,  0.0816,  0.1281,  0.0587,  0.0620,  0.0672, -0.0283,
        -0.0449,  0.0068,  0.0425,  0.0433, -0.0431, -0.0175,  0.1903,  0.1439,
         0.0423,  0.1799,  0.0559,  0.0837,  0.1213, -0.0369,  0.0048,  0.0142,
        -0.0038,  0.0619,  0.1502,  0.1371, -0.0536,  0.1634, -0.0078, -0.1081,
         0.0343,  0.1809,  0.0429,  0.1043, -0.0808,  0.0200, -0.0711,  0.1063,
         0.0110, -0.0026,  0.1414, -0.0803,  0.0826,  0.0426,  0.0773,  0.0549,
         0.0417,  0.0556, -0.0230, 

In [16]:
row = [24049,1541,49633,2610,80,6,264,5125872,8,7,1659,2514.0,467,0,207.375,221.2780652,1047,0,359.14285710000007,420.18386,814.0,2.0,366133.7143,1320784.35,4954614.0,73.0,171258.0,24465.42857,21945.246219999997,57169.0,1001.0,5125743.0,854290.5,2033515.304,5004981.0,2049.0,0,0,0,0,172,152,1.560710061,1.3656213030000002,0,1047,260.8125,322.5548468,104041.6292,0,0,0,1,0,0,0,0,0,278.2,207.375,359.14285710000007,0,0,0,0,0,0,8,1659,7,2514,8192,262,7,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
row = [row[:37]+row[38:39]+row[40:61]+row[67:]]
yhat = predict(row, model)
print(yhat)

[[[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]]
